

# AWS_reviews_count_vectorizer_sentiment_analysis


# Importing necessary libraries

In [ ]:
# Importing necessary libraries
import pandas as pd   # pandas for data manipulation and analysis
import numpy as np    # numpy for numerical operations
import os             # os for interacting with the operating system (e.g., file paths)
import re             # re for regular expression operations
import random         # random for generating random numbers


# loading the file

In [ ]:
# Reading the CSV file in chunks
# 'chunksize=10000' specifies that the file will be read in parts of 10,000 rows each
chunks = pd.read_csv("/content/aws_review_sofware_dataset.csv", sep=',', chunksize=10000)

# Get the first chunk of data from the iterator
df = next(chunks)

# Print the column names of the first chunk to understand the structure of the data
print(df.columns)


Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')


In [ ]:
df.columns

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')

In [ ]:
# Import the Natural Language Toolkit (nltk) library
import nltk

# Download the 'punkt' tokenizer models for tokenizing text
# This model is used for splitting text into sentences or words
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Initialize columns with default values and update to empty lists


In [ ]:

df["words"] = "default value"  # Set all entries in the 'words' column to "default value"
df["sentences"] = "default value"  # Set all entries in the 'sentences' column to "default value"
for i in range(df.shape[0]):  # 'df.shape[0]' gives the number of rows in the DataFrame
  df.at[i, "words"] = list("")  # 'df.at[i, "words"]' accesses the 'words' column of the ith row and assigns an empty list
  df.at[i, "sentences"] = list("")  # 'df.at[i, "sentences"]' accesses the 'sentences' column of the ith row and assigns an empty list


# sentence tokenization

In [1]:
# Import the 'sent_tokenize' function from the Natural Language Toolkit (nltk)
# This function is used to split text into sentences
from nltk.tokenize import sent_tokenize


In [ ]:
# Iterate over each row in the DataFrame
for i in range(df.shape[0]):
    # Tokenize the 'reviewText' of the current row into sentences
    l1 = sent_tokenize(str(df.loc[i, "reviewText"]))
    # Update the 'sentences' column of the current row with the tokenized sentences
    df.at[i, "sentences"] = l1


In [ ]:
# Install the 'pywsd' library using pip
# This library is used for word sense disambiguation and other NLP tasks
!pip install pywsd


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 64.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 12.4 MB/s eta 0:00:00
  Created wheel for wn: filename=wn-0.0.23-py3-none-any.whl size=31792912 sha256=d5c7ac8e5ead64a9def7ede814e49e4feeedafb3e6db864049965eb26afcfc41
  Stored in directory: /root/.cache/pip/wheels/a1/1a/7d/23a76ce45998af60e47466a694c237fa26023c5674b47672b2
Successfully built wn


In [ ]:
# Import the Natural Language Toolkit (nltk) library
import nltk

# Download the 'averaged_perceptron_tagger' model from NLTK
# This model is used for part-of-speech tagging in text
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
# Import the Natural Language Toolkit (nltk) library
import nltk

# Download the 'wordnet' lexical database from NLTK
# WordNet is used for lexical relationships and word sense disambiguation
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Import the 'lemmatize_sentence' function from the 'pywsd.utils' module
# This function is used to lemmatize all words in a sentence
from pywsd.utils import lemmatize_sentence


In [ ]:
# Import the Natural Language Toolkit (nltk) library
import nltk

# Download the 'wordnet' lexical database from NLTK
# This resource is used for understanding word meanings and relationships
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# lemmatization of sentence

In [ ]:
# Iterate over each row in the DataFrame
for k in range(df.shape[0]):
    # Initialize the 'words' column for the current row as an empty list
    df.at[k, "words"] = list("")

    # Iterate over each sentence in the 'sentences' column of the current row
    for j in range(len(df.loc[k, "sentences"])):
        # Lemmatize the current sentence and extend the 'words' list with the lemmatized words
        df.at[k, "words"].extend(lemmatize_sentence(df.loc[k, "sentences"][j]))


In [ ]:
# Initialize the 'words_sentences' column in the DataFrame with a default value
# All entries in this column are set to the string "default"
df["words_sentences"] = "default"


In [ ]:
# Import the 'functools' library for using higher-order functions
import functools

# Iterate over each row in the DataFrame
for k in range(df.shape[0]):
    # Combine the list of words in the 'words' column into a single string
    # Use functools.reduce to concatenate words with a space separator
    df.loc[k, "words_sentences"] = functools.reduce(
        lambda a, b: str(a) + " " + str(b),  # Lambda function to concatenate two strings with a space
        df.loc[k, "words"]  # List of words to concatenate
    )


#  CountVectorizer for converting text to a matrix of token counts

In [ ]:
# Import the 'CountVectorizer' class from Scikit-learn's text feature extraction module
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
# Create a copy of the DataFrame
df1 = df

# Define the number of features (words) to keep in the matrix
no_features = 1000

# Initialize the CountVectorizer with a maximum number of features and English stop words
tf_vectorizer = CountVectorizer(max_features=no_features, stop_words='english')

# Fit the CountVectorizer to the 'words_sentences' column and transform the text data into a feature matrix
tf = tf_vectorizer.fit_transform(df1.words_sentences)


In [ ]:
# Convert the sparse matrix returned by CountVectorizer into a DataFrame
df_x=pd.DataFrame(tf.A, columns=tf_vectorizer.get_feature_names_out())

In [ ]:
# Extract the 'verified' column from the DataFrame and assign it to df_y
df_y = df["verified"]


# LabelEncoder

In [ ]:
# Import the LabelEncoder class from Scikit-learn's preprocessing module
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Create an instance of the LabelEncoder class
# This object will be used to encode categorical labels as numerical values
le = LabelEncoder()


In [ ]:
# Convert the 'df_y' Series into a DataFrame
# This allows for easier manipulation and integration with other DataFrames
df_y_1 = pd.DataFrame(df_y)


In [ ]:
# Apply label encoding to each column in the DataFrame 'df_y_1'
# 'le.fit_transform' converts categorical labels into numerical values

In [ ]:
# Display the column names of the DataFrame 'df_y_enc'
df_y_enc.columns

Index(['verified'], dtype='object')

In [ ]:
# Display the first 5 rows of the DataFrame 'df_y_enc'
df_y_enc.head(5)

verified
0         1
1         1
2         1
3         1
4         0

#  GradientBoosting

In [ ]:
# Import the GradientBoostingClassifier class from Scikit-learn's ensemble module
# This class is used to perform classification using gradient boosting
from sklearn.ensemble import GradientBoostingClassifier


In [ ]:
# Create an instance of the GradientBoostingClassifier with 3000 boosting stages
# 'n_estimators=3000' specifies the number of boosting stages (trees) to be used in the model
GBC = GradientBoostingClassifier(n_estimators=3000)


In [ ]:
# Fit the GradientBoostingClassifier model to the feature matrix 'df_x' and target labels 'df_y_enc'
# This trains the model using the provided data
gb_c = GBC.fit(df_x, df_y_enc)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# Evaluate the performance of the trained GradientBoostingClassifier model
# 'score' method calculates the accuracy of the model on the feature matrix 'df_x' and target labels 'df_y_enc'
# Returns the accuracy score as a proportion of correctly classified samples
GBC.score(df_x, df_y_enc)


0.9499

# RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(df_x,df_y_enc)

# Accuracy
accuracy_rf = rf.score(df_x,df_y_enc)
print(f"Random Forest Accuracy: {accuracy_rf * 100:.2f}%")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest Accuracy: 99.93%


# naive_bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(df_x,df_y_enc)

# Compute accuracy
accuracy_nb = nb.score(df_x,df_y_enc)
print(f"Naive Bayes Accuracy: {accuracy_nb * 100:.2f}%")

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Naive Bayes Accuracy: 70.71%
